In [2]:
import pandas as pd
import requests
import json
from pprint import pprint
from config import openWeatherKey

In [4]:
weatherDisasterCSV = "Resources/disaster_data.csv"
coordCSV = "output_data/NEW_coordinates.csv"
wDisaster_df = pd.read_csv(weatherDisasterCSV)
desiredColumns = ["Dis No", "Year", "Seq", "Disaster Subgroup", "Disaster Type", "Disaster Subtype", "Disaster Subsubtype", "Entry Criteria", "ISO", "Location", "Start Month", "End Month", "Total Deaths", "No Injured", "No Affected", "No Homeless", "Total Affected", "Insured Damages ('000 US$)", "Total Damages ('000 US$)", "CPI"]
wDisaster_df = wDisaster_df.loc[wDisaster_df["ISO"] == "USA", desiredColumns]
wDisaster_df
pd.options.display.max_rows = 999

In [6]:
wDisaster_df

,Dis No,Year,Seq,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Entry Criteria,ISO,Location,Start Month,End Month,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Insured Damages ('000 US$),Total Damages ('000 US$),CPI
2,2000-0080-USA,2000,80,Hydrological,Flood,Riverine flood,NaN,Affect,USA,"Kentucky, Ohio provinces",2,2.0,3.0,NaN,231.0,NaN,231.0,NaN,NaN,67.355759
3,2000-0500-USA,2000,500,Hydrological,Flood,Riverine flood,NaN,Affect,USA,"Morris, Sussex districts (New Jersey province)",8,8.0,NaN,NaN,175.0,NaN,175.0,NaN,166000.0,67.355759
4,2000-0919-USA,2000,919,Meteorological,Storm,NaN,NaN,SigDam,USA,"Alabama, Georgia, Louisiana, North Carolina, S...",1,1.0,4.0,NaN,NaN,NaN,NaN,280000.0,350000.0,67.355759
5,2000-9712-USA,2000,9712,Climatological,Drought,Drought,NaN,Affected,USA,Wyoming province,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.355759
6,2000-0838-USA,2000,838,Meteorological,Storm,Convective storm,Winter storm/Blizzard,Kill,USA,"Texas, Oklahoma, New Mexico, Arkansas, Missour...",12,12.0,57.0,34.0,NaN,NaN,34.0,200500.0,10000.0,67.355759
7,2000-0021-USA,2000,21,Meteorological,Storm,Convective storm,Tornado,Affect,USA,"Crittenden, Daviess, Webster districts (Kentuc...",1,1.0,1.0,12.0,NaN,150.0,162.0,200500.0,210000.0,67.355759
8,2000-0067-USA,2000,67,Meteorological,Storm,Convective storm,Tornado,Kill,USA,"Bullock, Montgomery districts (Alabama provinc...",2,2.0,22.0,100.0,900.0,NaN,1000.0,NaN,NaN,67.355759
9,2000-0175-USA,2000,175,Meteorological,Storm,Convective storm,Tornado,Affect,USA,"Fort Worth Texas city (Tarrant district, Texas...",3,3.0,5.0,NaN,3000.0,300.0,3300.0,NaN,450000.0,67.355759
10,2000-0338-USA,2000,338,Meteorological,Storm,Convective storm,Tornado,Declar/Int,USA,"Adams, Columbia, Crawford, Dane, Dodge, Grant,...",5,5.0,NaN,NaN,NaN,NaN,NaN,NaN,28500.0,67.355759
11,2000-0463-USA,2000,463,Meteorological,Storm,Convective storm,Tornado,Affect,USA,"Granite Falls city (Yellow Medicine, Minnesota...",7,7.0,1.0,15.0,900.0,120.0,1035.0,NaN,8100.0,67.355759


In [24]:
newData = {
    "Location": [],
    "Dis No": []
}

for index, row in wDisaster_df.iterrows():
    #print(type(newData["Location"]))
    if type(row["Location"]) is str:
        for location in row["Location"].split(","):
            newData["Location"].append(location)
            newData["Dis No"].append(row["Dis No"])
splitLocations_df = pd.DataFrame(newData)
splitLocations_df["Location"].value_counts()

 Missouri                                   80
 Kansas                                     77
 Oklahoma                                   60
 Indiana                                    59
 Mississippi                                58
                                            ..
 Winnebago districts (Illinois province)     1
 Calcasieu                                   1
 Kiowa                                       1
 Smith                                       1
 Commerce areas (Ottawa district             1
Name: Location, Length: 1415, dtype: int64

In [25]:
print(wDisaster_df.columns)
wDisaster_df.groupby("Disaster Type").count()

Index(['Dis No', 'Year', 'Seq', 'Disaster Subgroup', 'Disaster Type',
       'Disaster Subtype', 'Disaster Subsubtype', 'Entry Criteria', 'ISO',
       'Location', 'Start Month', 'End Month', 'Total Deaths', 'No Injured',
       'No Affected', 'No Homeless', 'Total Affected',
       'Insured Damages ('000 US$)', 'Total Damages ('000 US$)', 'CPI'],
      dtype='object')


,Dis No,Year,Seq,Disaster Subgroup,Disaster Subtype,Disaster Subsubtype,Entry Criteria,ISO,Location,Start Month,End Month,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Insured Damages ('000 US$),Total Damages ('000 US$),CPI
Disaster Type,,,,,,,,,,,,,,,,,,,
Drought,11,11,11,11,11,0,11,11,11,11,8,0,0,0,0,0,3,9,11
Earthquake,9,9,9,9,9,0,9,9,9,9,9,3,7,7,2,9,4,8,9
Epidemic,3,3,3,3,2,0,3,3,2,3,3,1,0,3,0,3,0,0,3
Extreme temperature,13,13,13,13,13,2,13,13,13,13,13,12,1,0,0,1,3,3,13
Flood,94,94,94,94,79,0,93,94,94,94,94,70,7,69,5,73,22,54,94
Landslide,3,3,3,3,3,0,3,3,3,3,3,3,2,1,1,3,0,2,3
Storm,278,278,278,278,258,203,278,278,278,278,278,238,102,86,29,155,175,224,274
Volcanic activity,1,1,1,1,1,0,1,1,1,1,1,0,0,1,0,1,0,1,1
Wildfire,62,62,62,62,52,0,62,62,62,62,62,32,22,26,40,54,17,42,62


In [26]:
splitLocations_df["Lat"] = ""
splitLocations_df["Long"] = ""
locations = splitLocations_df["Location"]
splitLocations_df

,Location,Dis No,Lat,Long
0,Kentucky,2000-0080-USA,,
1,Ohio provinces,2000-0080-USA,,
2,Morris,2000-0500-USA,,
3,Sussex districts (New Jersey province),2000-0500-USA,,
4,Alabama,2000-0919-USA,,
...,...,...,...,...
4019,Georgia,2020-0082-USA,,
4020,Texas,2020-0082-USA,,
4021,errebonne,2020-0219-USA,,
4022,Plaquemines,2020-0219-USA,,


In [7]:
coordLocations_df = pd.read_csv(coordCSV)
#This array is stored for potential future reference if adding new data from different dataframes
cleanedCoordsIndex = coordLocations_df.index
coordLocations_df

,Unnamed: 0,Unnamed: 0.1,Location,Dis No,Lat,Long
0,0,0,Kentucky,2000-0080-USA,34.61,-92.66
1,1,2,Morris,2000-0500-USA,40.83,-74.53
2,2,4,Alabama,2000-0919-USA,32.75,-86.75
3,3,5,Georgia,2000-0919-USA,32.75,-83.50
4,4,6,Louisiana,2000-0919-USA,31.00,-92.00
...,...,...,...,...,...,...
2856,2856,4016,Kentucky,2020-0082-USA,34.61,-92.66
2857,2857,4017,Missouri,2020-0082-USA,38.25,-92.50
2858,2858,4018,Mississippi,2020-0082-USA,32.75,-89.75
2859,2859,4019,Georgia,2020-0082-USA,32.75,-83.50


In [12]:
thirdQuintant = coordLocations_df.loc[(coordLocations_df["Long"] < -90) & (coordLocations_df["Long"] > -105), :]
thirdQuintant

,Unnamed: 0,Unnamed: 0.1,Location,Dis No,Lat,Long
0,0,0,Kentucky,2000-0080-USA,34.61,-92.66
4,4,6,Louisiana,2000-0919-USA,31.00,-92.00
11,11,15,Texas,2000-0838-USA,31.25,-99.25
12,12,16,Oklahoma,2000-0838-USA,35.49,-97.50
14,14,18,Arkansas,2000-0838-USA,34.75,-92.50
...,...,...,...,...,...,...
2851,2851,4009,Louisiana,2020-0011-USA,31.00,-92.00
2855,2855,4013,Kentucky,2020-0011-USA,34.61,-92.66
2856,2856,4016,Kentucky,2020-0082-USA,34.61,-92.66
2857,2857,4017,Missouri,2020-0082-USA,38.25,-92.50
